# DuckDB test notebook

This notebook is for testing and querying the DuckDB database after data ingestion.


In [7]:
import duckdb
import os
import sys
import pandas as pd

# Get the notebook's directory (tests folder)
notebook_dir = os.getcwd()
if 'exploration' in notebook_dir:
    # We're in exploration folder, go up one level
    project_root = os.path.dirname(notebook_dir)
else:
    # We're already in project root
    project_root = notebook_dir

# Add project root to Python path
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Change working directory to project root
os.chdir(project_root)

print(f"Notebook directory: {notebook_dir}")
print(f"Project root: {project_root}")
print(f"Current working directory: {os.getcwd()}")

# Try to import, with fallback
try:
    from ingestion.config import DUCKDB_PATH
    print(f"✓ Successfully imported DUCKDB_PATH: {DUCKDB_PATH}")
except ImportError as e:
    print(f"⚠ Import error: {e}")
    # Fallback: construct path manually
    DUCKDB_PATH = os.path.join(project_root, "duckdb", "football.duckdb")
    print(f"Using fallback path: {DUCKDB_PATH}")

# Connect to DuckDB
db_path = os.path.abspath(DUCKDB_PATH)
print(f"\nConnecting to DuckDB at: {db_path}")

# Check if file exists
if not os.path.exists(db_path):
    print(f"⚠ Warning: Database file does not exist at {db_path}")
    print("Make sure you've run the ingestion pipelines first!")
else:
    print(f"✓ Database file found")

conn = duckdb.connect(db_path)
print("✓ Connected successfully!")


Notebook directory: /Users/admin/Documents/github/PL_dashboard/exploration
Project root: /Users/admin/Documents/github/PL_dashboard
Current working directory: /Users/admin/Documents/github/PL_dashboard
✓ Successfully imported DUCKDB_PATH: /Users/admin/Documents/github/PL_dashboard/duckdb/football.duckdb

Connecting to DuckDB at: /Users/admin/Documents/github/PL_dashboard/duckdb/football.duckdb
✓ Database file found
✓ Connected successfully!


## Check Database Schema and Tables

In [8]:
# List all schemas
schemas = conn.execute("SELECT schema_name FROM information_schema.schemata").fetchall()
print("Schemas:")
for schema in schemas:
    print(f"  - {schema[0]}")


Schemas:
  - main
  - main_mart
  - main_src
  - main_stg
  - raw
  - information_schema
  - main
  - pg_catalog
  - main


In [9]:
# List all tables in raw schema
tables = conn.execute("""
    SELECT table_name 
    FROM information_schema.tables 
    WHERE table_schema = 'raw'
    ORDER BY table_name
""").fetchall()

print("Tables in 'raw' schema:")
for table in tables:
    print(f"  - {table[0]}")


Tables in 'raw' schema:
  - _dlt_loads
  - _dlt_pipeline_state
  - _dlt_version
  - fixtures
  - ingestion_audit
  - scorers
  - standings
  - teams


## Check Row Counts

In [10]:
# Get row counts for all tables
for table in tables:
    table_name = table[0]
    try:
        count = conn.execute(f"SELECT COUNT(*) FROM raw.{table_name}").fetchone()[0]
        print(f"{table_name}: {count} rows")
    except Exception as e:
        print(f"{table_name}: Error - {e}")

_dlt_loads: 4 rows
_dlt_pipeline_state: 4 rows
_dlt_version: 4 rows
fixtures: 170 rows
ingestion_audit: 4 rows
scorers: 20 rows
standings: 20 rows
teams: 20 rows


## Query Teams Table

In [12]:
 Get teams data
teams_df = conn.execute("""
    SELECT 
        *
    FROM raw.teams
    ORDER BY team_name
    LIMIT 10
""").df()

teams_df

SyntaxError: invalid syntax (2138932884.py, line 1)

In [13]:
print(teams_df['team_logo'][1])

NameError: name 'teams_df' is not defined

## Query Standings Table

In [15]:
# Get standings data
standings_df = conn.execute("""
    SELECT 
        rank,
        team_name,
        points,
        all_played,
        all_win,
        all_draw,
        all_lose,
        all_goals_for,
        all_goals_against,
        goals_diff
    FROM raw.standings
    ORDER BY rank
    LIMIT 21
""").df()

standings_df

,rank,team_name,points,all_played,all_win,all_draw,all_lose,all_goals_for,all_goals_against,goals_diff
0,1,Arsenal,39,17,12,3,2,31,10,21
1,2,Manchester City,37,17,12,1,4,41,16,25
2,3,Aston Villa,36,17,11,3,3,27,18,9
3,4,Chelsea,29,17,8,5,4,29,17,12
4,5,Liverpool,29,17,9,2,6,28,25,3
5,6,Sunderland,27,17,7,6,4,19,17,2
6,7,Manchester United,26,17,7,5,5,31,28,3
7,8,Crystal Palace,26,17,7,5,5,21,19,2
8,9,Brighton,24,17,6,6,5,25,23,2
9,10,Everton,24,17,7,3,7,18,20,-2


## Query Fixtures Table

In [16]:
# Get fixtures data
fixtures_df = conn.execute("""
    SELECT 
        *
    FROM raw.fixtures
    ORDER BY fixture_date DESC
    LIMIT 10
""").df()

fixtures_df


,fixture_id,fixture_date,fixture_timestamp,fixture_timezone,fixture_referee,venue_id,venue_name,venue_city,league_id,league_name,...,away_team_name,away_team_winner,home_goals,away_goals,score_halftime_home,score_halftime_away,score_fulltime_home,score_fulltime_away,_dlt_load_id,_dlt_id
0,1379133,2025-12-22 21:00:00+01:00,1766433600,UTC,None,535,Craven Cottage,London,39,Premier League,...,Nottingham Forest,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1766429582.648793,GigfiyryExwdBA
1,1379129,2025-12-21 17:30:00+01:00,1766334600,UTC,"Michael Oliver, England",495,Villa Park,Birmingham,39,Premier League,...,Manchester United,False,2,1,1,1,2,1,1766429582.648793,azgUuNr01M7bqw
2,1379132,2025-12-20 21:00:00+01:00,1766260800,UTC,"Samuel Barrott, England",22033,Hill Dickinson Stadium,Liverpool,39,Premier League,...,Arsenal,True,0,1,0,1,0,1,1766429582.648793,ydF7qfm/f/8BTA
3,1379134,2025-12-20 21:00:00+01:00,1766260800,UTC,"Thomas Bramall, England",546,Elland Road,Leeds,39,Premier League,...,Crystal Palace,False,4,1,2,0,4,1,1766429582.648793,7FyITOqV0H8uEA
4,1379137,2025-12-20 18:30:00+01:00,1766251800,UTC,"John Brooks, England",593,Tottenham Hotspur Stadium,London,39,Premier League,...,Liverpool,True,1,2,0,0,1,2,1766429582.648793,DvaBMH5hyfujaQ
5,1379130,2025-12-20 16:00:00+01:00,1766242800,UTC,"Chris Kavanagh, England",504,Vitality Stadium,Bournemouth,39,Premier League,...,Burnley,<NA>,1,1,0,0,1,1,1766429582.648793,E/WecxYQZXd/2Q
6,1379138,2025-12-20 16:00:00+01:00,1766242800,UTC,"Matt Donohue, England",600,Molineux Stadium,Wolverhampton,39,Premier League,...,Brentford,True,0,2,0,0,0,2,1766429582.648793,wiKGNh63dLH7rg
7,1379135,2025-12-20 16:00:00+01:00,1766242800,UTC,"Paul Tierney, England",555,Etihad Stadium,Manchester,39,Premier League,...,West Ham,False,3,0,2,0,3,0,1766429582.648793,lgQZyu1729WRSw
8,1379131,2025-12-20 16:00:00+01:00,1766242800,UTC,"Darren England, England",508,Amex Stadium,Brighton,39,Premier League,...,Sunderland,<NA>,0,0,0,0,0,0,1766429582.648793,2x1YAOP7TdOGYQ
9,1379136,2025-12-20 13:30:00+01:00,1766233800,UTC,"Andy Madley, England",562,St. James' Park,Newcastle,39,Premier League,...,Chelsea,<NA>,2,2,2,0,2,2,1766429582.648793,jkZy7u0r40wg0Q


## Query Top Scorers Table

In [17]:
# Get top scorers data
scorers_df = conn.execute("""
    SELECT 
        *
    FROM raw.scorers
    ORDER BY goals_total DESC
    LIMIT 10
""").df()

scorers_df

,player_id,player_name,player_firstname,player_lastname,player_age,player_birth_date,player_birth_place,player_birth_country,player_nationality,player_height,...,games_minutes,games_number,games_position,games_rating,games_captain,goals_total,goals_assists,goals_conceded,_dlt_load_id,_dlt_id
0,1100,E. Haaland,Erling,Braut Haaland,25,2000-07-21,Leeds,England,Norway,195,...,1466,9,Attacker,7.78,False,19,4,0,1766429584.506816,zdI8AxiC5ayooA
1,196156,Thiago,Igor Thiago,Nascimento Rodrigues,24,2001-06-26,Gama,Brazil,Brazil,191,...,1403,9,Attacker,6.97,False,11,0,0,1766429584.506816,bIQM8loKpjEVMQ
2,174565,H. Ekitike,Hugo,Ekitike,23,2002-06-20,Reims,France,France,190,...,961,22,Attacker,6.91,False,8,1,0,1766429584.506816,faHE8GPKdbEnFw
3,19281,A. Semenyo,Antoine Serlom,Semenyo,25,2000-01-07,London,England,Ghana,185,...,1439,24,Midfielder,7.26,False,8,3,0,1766429584.506816,90TYJZxjxbAOmA
4,19170,M. Rogers,Morgan Elliot,Rogers,23,2002-07-26,Halesowen,England,England,187,...,1518,27,Midfielder,7,False,7,3,0,1766429584.506816,i97CNVqWDF885A
5,2413,Richarlison,Richarlison,de Andrade,28,1997-05-10,Nova Venécia,Brazil,Brazil,184,...,985,9,Attacker,6.86,False,7,2,0,1766429584.506816,w8tASqZIy5BQ6Q
6,631,P. Foden,Philip Walter,Foden,25,2000-05-28,Stockport,England,England,171,...,1227,47,Midfielder,7.31,False,7,2,0,1766429584.506816,Jis0CQhctWuO5Q
7,1469,D. Welbeck,Daniel Nii,Tackie Mensah Welbeck,35,1990-11-26,Manchester,England,England,185,...,988,18,Attacker,6.76,False,7,0,0,1766429584.506816,+KnZoFbp37jBzw
8,158054,N. Woltemade,Nick,Woltemade,23,2002-02-14,Bremen,Germany,Germany,198,...,1102,27,Attacker,6.76,False,7,1,0,1766429584.506816,5SL9nnvUpUghDQ
9,18766,D. Calvert-Lewin,Dominic Nathaniel,Calvert-Lewin,28,1997-03-16,Sheffield,England,England,187,...,1032,9,Attacker,6.77,False,7,0,0,1766429584.506816,hx02rTfEv7q/sw


In [18]:
# Close the connection
conn.close()
print("Connection closed.")


Connection closed.
